In [2]:
import background_generation as backgen
import numpy as np
import pickle

def return_background(generate,dir_images,dir_labels,directory_backs,vector_backgrounds,img_back):
    if generate:
        while not img_back in vector_backgrounds:
            if img_back in dir_images and img_back in dir_labels:
                background = backgen.estimate_background(f'{dir_images}/{img_back}', f'{dir_labels}/{img_back}')
            else:
                n_background = np.random.randint(0, len(vector_backgrounds))
                img_back = vector_backgrounds[n_background]
                background = np.array(Image.open(f'{directory_backs}/{vector_backgrounds[n_background]}')) 
    else:
        n_background = np.random.randint(0, len(vector_backgrounds))
        img_back = vector_backgrounds[n_background]
        background = np.array(Image.open(f'{directory_backs}/{img_back}'))
    return background 


def returns_norms_pickles(num_maps, dir_maps_pickle, background, vector_maps):
    
    sorted_array_pickels = []
    for i in range(num_maps):
        normalized_original_map = None    
        while normalized_original_map is None:
            n_random_map = np.random.randint(0, len(vector_maps))
            path_map = (f"{dir_maps_pickle}/{vector_maps[n_random_map]}")
            map_pickle = pickle.load(open(path_map, 'rb'))           
            vessel_map = map_pickle['vessel_model'].vessel_map 
            original_map = vessel_map.mapped_values
            vessel_mask = vessel_map.mapped_mask_values
            normalized_original_map = backgen.normalize(background,original_map,vessel_mask,30)            
            if normalized_original_map is not None:
                sorted_array_pickels.append(vector_maps[n_random_map])
    return sorted_array_pickels

In [23]:
#Usar os mesmos mapas sempre
from pathlib import Path
import numpy as np
from PIL import Image
import sys
from matplotlib import pyplot as plt

import background_generation as backgen
#windows
sys.path.insert(0, r"C:\Users\adria\Documents\Mestrado\texture_codes\modules")
root_dir = Path(r"C:\Users\adria\Documents\Mestrado\texture_codes\modules")

#linux
#sys.path.insert(0, "/home/adriano/projeto_mestrado/modules/")
#root_dir = f"/home/adriano/projeto_mestrado/modules"


from Utils import functions
import pickle

params = { 'dir_maps_pickle': f'{root_dir}/Vessel_models_pickle',
            'num_maps': 5, # number of maps to be inserted
            'num_images': 100, #number of images desired
            'dir_backs': f'{root_dir}/Background/Artificially_generated_maps', #background's directory
            'dir_images': f'{root_dir}/Images/vessel_data/images', # original images directory
            'dir_labels': f'{root_dir}/Images/vessel_data/labels_20x', # label directory
            'dir_traces': f'{root_dir}/Artificial_lines/bezier_traces', # directory of traces - Bezier curves
            'generate_back':False, #
            'out_dir': f'{root_dir}/Images/Background_with_vessels_tests', #output directory
            'min_number_vessels': 20, # minimum number of vessels
            'max_number_vessels': 50, # maximum number of vessels

          }

#def generate_maps(params):
array_maps = functions.read_directories(params['dir_maps_pickle'])
array_images = functions.read_directories(params['dir_images'])
array_labels = functions.read_directories(params['dir_labels'])
array_backrounds = functions.read_directories(params['dir_backs'])
array_traces = functions.read_directories(params['dir_traces'])

returns_norms_pickles
none_results = 0
counter = 0
for j in range(3):
    number_of_vessels = np.random.randint(params['min_number_vessels'], params['max_number_vessels']) 
    
    n_random = np.random.randint(0, len(array_images))
    path_background = array_images[n_random]
    
    background = return_background(params['generate_back'],params["dir_images"],params["dir_labels"],params['dir_backs'],array_backrounds, path_background)

    #array_itens = returns_ves_norm_binary(params['num_maps'],params['dir_maps_pickle'],background, array_maps)

    background_name = f'{path_background}'
    background_name = background_name.replace("'","").replace(".tiff","")

    clipping_background = background[0:1100,0:1370]
    background_with_pad = np.pad(clipping_background, ((200,200),(200,200)), mode="symmetric", reflect_type="even")
    background_bin = np.zeros(background_with_pad.shape)

    background_with_vessels_bin = background_bin.copy()
    background_with_vessels = background_with_pad.copy()

    has_maps =  np.full(shape = background_with_pad.shape, fill_value=0)
    has_maps_bin =  np.full(shape = background_bin.shape, fill_value=0)

    while counter < number_of_vessels:
        n_traces = np.random.randint(0, len(array_traces))
        trace = array_traces[n_traces]    
        vector_medial_path = backgen.return_paths(f"{params['dir_traces']}/{trace}")
        n_random = np.random.randint(0, len(array_itens))
        vessel_map = array_itens[n_random]['vessel_map']
        normalized_original_map = array_itens[n_random]['normalized_original_map']
        binary_map_without_artifacts = array_itens[n_random]['binary_map_without_artifacts']
        results = backgen.insert_vessels2(vector_medial_path[0], vector_medial_path[1], vessel_map, normalized_original_map, binary_map_without_artifacts, background)
        if results is not None:
            vessel_without_artifacts, map_without_artifacts, mask_map, treshold = results  
            background_with_vessels = backgen.insert_map(background_with_vessels,vessel_without_artifacts,map_without_artifacts,mask_map, treshold, has_maps)
            background_with_vessels_bin = backgen.insert_binary_map(background_with_vessels_bin,vessel_without_artifacts,has_maps_bin)
            counter +=1
        else:
            none_results += 1  

    background_clipped = background_with_vessels[200:1304,200:1576]
    background_clipped_bin = background_with_vessels_bin[200:1304,200:1576]

    img1 = Image.fromarray(background_clipped.astype(np.uint8))
    path = f"{params['out_dir']}/images/{background_name}_{j}_com_{number_of_vessels}.tiff"
    img = img1.save(path)

    img2 = Image.fromarray(background_clipped_bin.astype(np.bool_))
    path = f"{params['out_dir']}/labels/{background_name}_{j}_com_{number_of_vessels}.tiff"
    img = img2.save(path)
print(f'resultados_none: {none_results}')

resultados_none: 149


In [ ]:
plt.figure(figsize=[10, 8])
plt.title("fundo_com_vasos")
plt.imshow(background, 'gray', vmin=0, vmax=255)
plt.plot()